In [8]:
import numpy as np 
import pandas as pd
from scipy import stats
import seaborn as sns 
import matplotlib.pyplot as plt

In [9]:
urlfhh = "2022CMS/Citywide_Mobility_Survey_-_Household_2032.csv"
dfhh = pd.read_csv(urlfhh)

urlday = "2022CMS/Citywide_Mobility_Survey_-_Day_2022.csv"
urlhhl = "2022CMS/Citywide_Mobility_Survey_-_Household_2022.csv"
urlper = "2022CMS/Citywide_Mobility_Survey_-_Person_2022.csv"
urlzon = "2022CMS/Citywide_Mobility_Survey_-_Survey_Zones.csv"
urltrp = "2022CMS/Citywide_Mobility_Survey_-_Trip_2022.csv"
urlveh = "2022CMS/Citywide_Mobility_Survey_-_Vehicle_2022.csv"

#### Read the .csv file using `pandas` libriary and store it as a `pandas` Data Frame
dday = pd.read_csv(urlday)
dhhl = pd.read_csv(urlhhl)
dper = pd.read_csv(urlper)
dzon = pd.read_csv(urlzon)
dtrp = pd.read_csv(urltrp)
dveh = pd.read_csv(urlveh)

In [10]:
#Merge
dcfm1 = pd.merge(dfhh, dtrp, on = ['hh_id'])
dcfm1.head(10) #Check

,hh_id,hh_cms_zone_gp,participation_group,signup_platform,diary_platform,num_days_complete,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,...,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight
0,22000050,1,9,rmove,rmove,5,4,1,1,1,...,0.000000,0,0,2,3,0,1,4,0.000000,215.724068
1,22000050,1,9,rmove,rmove,5,4,1,1,1,...,5856.666667,0,0,3,2,0,4,1,0.000000,215.724068
2,22000050,1,9,rmove,rmove,5,4,1,1,1,...,14.716667,0,0,2,3,0,1,4,359.540113,215.724068
3,22000050,1,9,rmove,rmove,5,4,1,1,1,...,431.733333,0,0,3,2,0,4,1,359.540113,215.724068
4,22000105,1,9,rmove,rmove,7,5,2,1,1,...,127.200000,0,0,3,3,0,2,2,861.630290,492.360166
5,22000105,1,9,rmove,rmove,7,5,2,1,1,...,0.000000,0,0,3,3,0,2,4,861.630290,492.360166
6,22000105,1,9,rmove,rmove,7,5,2,1,1,...,0.000000,0,0,3,3,0,4,4,861.630290,492.360166
7,22000105,1,9,rmove,rmove,7,5,2,1,1,...,0.000000,0,0,3,4,1,4,4,861.630290,492.360166
8,22000105,1,9,rmove,rmove,7,5,2,1,1,...,9.600000,0,0,4,3,1,4,4,861.630290,492.360166
9,22000105,1,9,rmove,rmove,7,5,2,1,1,...,0.000000,0,0,3,2,1,4,4,861.630290,492.360166


In [11]:
dcfm2 = dcfm1.drop(dcfm1[(dcfm1['income_broad'] > 8)].index) #Removing Missing Responses
dcfm3 = dcfm2.drop(dcfm2[(dcfm2['num_vehicles'] == 995)].index) #Removing Missing Responses

dcfm4 = dcfm3.drop(dcfm3[(dcfm3['travel_dow'] > 5)].index) #Removing Weekends
dcfm = dcfm4.drop(dcfm4[(dcfm4['d_cms_zone'] == 11)].index) #Removing Outside of New York

#Assign Unique Values
arrayzone = dcfm['d_cms_zone'].unique()
arrayhhinc = dcfm['income_broad'].unique()
arraynumw = dcfm['num_workers'].unique()
arrayveh = dcfm['num_vehicles'].unique()

ccmcurrent = dcfm.groupby(['d_cms_zone','income_broad', 'num_workers', 'num_vehicles']).size()
print(ccmcurrent) #Check
ccmhhcurrent = dcfm.groupby(['d_cms_zone','income_broad', 'num_workers', 'num_vehicles','hh_id']).size()

ccmnew = dcfm.groupby(['d_cms_zone','new_income_broad', 'new_num_workers', 'new_num_vehicles']).size()
print(ccmnew) #Check
ccmhhnew = dcfm.groupby(['d_cms_zone','new_income_broad', 'new_num_workers', 'new_num_vehicles', 'hh_id']).size()

d_cms_zone  income_broad  num_workers  num_vehicles
1           1             0            0               312
                                       1                86
                                       3                19
                          1            0                88
                                       1                60
                                                      ... 
10          6             5            4                20
            7             0            2                25
                          1            2                 8
                          2            1                24
                                       2                29
Length: 792, dtype: int64
d_cms_zone  new_income_broad  new_num_workers  new_num_vehicles
1           1                 0                0                   388
                                               1                   152
                                               3                   

In [12]:
totalnewtrips = 0
countmisshh = 0
totalnewtrips_zone = 0
for keyzone in arrayzone:
    print("Zone", keyzone)
    for keyinc in arrayhhinc:
        for keywrk in arraynumw:
            for keyveh in arrayveh:
                    try:
                        average = ccmcurrent[keyzone, keyinc, keywrk, keyveh]/ccmhhcurrent[keyinc, keywrk, keyveh].size
                        totalnewtrips += ccmhhnew[keyzone, keyinc, keywrk, keyveh].size * average
                        totalnewtrips_zone += ccmhhnew[keyzone, keyinc, keywrk, keyveh].size * average
                        print("Total Future Number of Trips: ", totalnewtrips_zone)
                        totalnewtrips_zone = 0
                    except:
                        countmisshh += 1 # This count allows us to know, how many groups in the future will not have inf in current data

print ("Number of HH groups without data from current time period ", countmisshh)
print ("Total number of future trips ", totalnewtrips)

Zone 3
Total Future Number of Trips:  67.5
Total Future Number of Trips:  355.25
Total Future Number of Trips:  14.303571428571427
Total Future Number of Trips:  70.5
Total Future Number of Trips:  0.0625
Total Future Number of Trips:  0.23728813559322035
Total Future Number of Trips:  8.0
Total Future Number of Trips:  18.9
Total Future Number of Trips:  0.8695652173913043
Total Future Number of Trips:  1.8764044943820224
Total Future Number of Trips:  14.727272727272727
Total Future Number of Trips:  2.24
Total Future Number of Trips:  25.548387096774196
Total Future Number of Trips:  372.48275862068965
Total Future Number of Trips:  7.829787234042553
Total Future Number of Trips:  52.10526315789473
Total Future Number of Trips:  0.09090909090909091
Total Future Number of Trips:  0.3673469387755102
Total Future Number of Trips:  73.0
Total Future Number of Trips:  106.0
Total Future Number of Trips:  244.93333333333334
Total Future Number of Trips:  745.8000000000001
Total Future Num